In [8]:
import quinn
from kaggle.api.kaggle_api_extended import KaggleApi
from pyspark.sql import SparkSession
from datetime import date
from utils import dashes_to_underscores

spark = SparkSession.builder.getOrCreate()

dataset_df = spark.read.csv('./data/Monkey_Pox_Cases_Worldwide.csv', header=True)
daily_df = spark.read.csv('./data/Daily_Country_Wise_Confirmed_Cases.csv', header=True)


total_df = dataset_df.select(dataset_df['Country'], dataset_df['Confirmed_Cases'])

dataset_df = total_df.join(daily_df,['Country'],how='left')

dataset_df = dataset_df.transform(quinn.with_columns_renamed(dashes_to_underscores))

dataset_df = daily_df.transform(quinn.with_columns_renamed(dashes_to_underscores))

dataset_df.groupBy("Country").pivot("Country")

pandas_DF = dataset_df.toPandas()

pandas_DF.set_index('Country', inplace=True)
transpose_df = pandas_DF.transpose()
transpose_df.rename(columns = {'Country':'Date'})

transpose_df.reset_index(inplace=True)

spark_df=spark.createDataFrame(transpose_df)
spark_df = spark_df.withColumnRenamed('index', 'Date')

spark_df.show()

+----------+-------+--------+-----+-------------+-------+-------+------+-----+------+------+---------+-----------+------+-----------+-------+--------+-------+--------------+--------------------+--------+-----+----------------+---------+-------+-------+-----+------+------+-------+---------+-------+------+------+------+-----+------+---------+-------+-----+-------+-------+----------+-----+------+---------+-------+------------+-----------+--------+-------+--------+------+----+-------+-----------+------+
|      Date|England|Portugal|Spain|United States|Germany|Belgium|Sweden|Italy|Canada|France|Australia|Netherlands|Israel|Switzerland|Austria|Scotland|Denmark|Czech Republic|United Arab Emirates|Slovenia|Wales|Northern Ireland|Argentina|Ireland|Finland|Malta|Mexico|Norway|Hungary|Gibraltar|Morocco|Latvia|Greece|Brazil|Ghana|Poland|Venezuela|Romania|Benin|Iceland|Georgia|Luxembourg|Chile|Serbia|Singapore|Lebanon|South Africa|South Korea|Colombia|Croatia|Bulgaria|Taiwan|Peru|Estonia|Puerto Ric

In [40]:
from utils import date_check, date_minus_weeks
import pyspark.sql.functions as sf

# spark_df.select('Date','United States').where(spark_df['United States'] >= 1).show()

spark_df.select('*').show()

+----------+-------+--------+-----+-------------+-------+-------+------+-----+------+------+---------+-----------+------+-----------+-------+--------+-------+--------------+--------------------+--------+-----+----------------+---------+-------+-------+-----+------+------+-------+---------+-------+------+------+------+-----+------+---------+-------+-----+-------+-------+----------+-----+------+---------+-------+------------+-----------+--------+-------+--------+------+----+-------+-----------+------+
|      Date|England|Portugal|Spain|United States|Germany|Belgium|Sweden|Italy|Canada|France|Australia|Netherlands|Israel|Switzerland|Austria|Scotland|Denmark|Czech Republic|United Arab Emirates|Slovenia|Wales|Northern Ireland|Argentina|Ireland|Finland|Malta|Mexico|Norway|Hungary|Gibraltar|Morocco|Latvia|Greece|Brazil|Ghana|Poland|Venezuela|Romania|Benin|Iceland|Georgia|Luxembourg|Chile|Serbia|Singapore|Lebanon|South Africa|South Korea|Colombia|Croatia|Bulgaria|Taiwan|Peru|Estonia|Puerto Ric

In [38]:
spark_df.select('Date').where(spark_df.columns == 1).show()

TypeError: condition should be string or Column